In [1]:
%load_ext autoreload
%autoreload 2
from fastai import *
from fastai.text import *
from fastai.callbacks import *
import numpy as np
import spacy

In [2]:
DATA_PATH = "/Users/Reagan/Dropbox/SF_DS/practicum/wolof/fastai/data"
EXTR_PATH = pathlib.Path(f"{DATA_PATH}/wiki_extr/wo")

In [3]:
LM_PATH = Path(f"{DATA_PATH}/model/wolooof/")
LM_PATH.mkdir(parents=True, exist_ok=True)

In [4]:
lang = 'wo'

# build pre-trained model

In [121]:
bs = 64
data= TextLMDataBunch.from_csv(LM_PATH, 'wolof_types.csv')
data.save('data_export.pkl')
data = load_data(LM_PATH, 'data_export.pkl', bs=bs)

In [122]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5, pretrained=False)

In [124]:
lr = 1e-2
lr *= bs/48 
learn.unfreeze()
learn.fit_one_cycle(10, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,6.317068,5.828072,0.137688,27:51
1,5.282965,4.920443,0.213662,26:39
2,4.738628,4.559748,0.239494,26:14
3,4.386663,4.336269,0.260085,25:59
4,4.044110,4.134867,0.284852,25:56
5,3.690162,3.933771,0.315579,26:07
6,3.330341,3.740444,0.348864,25:57
7,2.987522,3.594601,0.376639,25:56
8,2.727347,3.528127,0.390787,26:34
9,2.612818,3.521239,0.392680,27:17


# save pre-trained model and vocab

In [125]:
# save model and vocab
mdl_path = LM_PATH/'models'
mdl_path.mkdir(exist_ok=True)
lm_fns = [f'{lang}_wt', f'{lang}_wt_vocab']
learn.to_fp32().save(mdl_path/lm_fns[0], with_opt=False)
learn.data.vocab.save(mdl_path/(lm_fns[1] + '.pkl'))

# fine-tune on new data

In [ ]:
geoall = pd.read_csv(LM_PATH/'wiki_geo.csv')[['text','geo']]
train_df, test_df = train_test_split(geoall, test_size=0.2)
data_lm = (TextList.from_df(geoall, LM_PATH, cols='text')
        .split_by_rand_pct(0.1, seed=42)
        .label_for_lm()           
        .databunch(bs=bs, num_workers=1))

In [5]:
from sklearn.model_selection import train_test_split

geoall = pd.read_csv(LM_PATH/'wiki_geo.csv')[['text','geo']]
train_df, test_df = train_test_split(geoall, test_size=0.2)

In [6]:
print(288/1075)
print(78/269)

0.26790697674418606
0.2899628252788104


In [11]:
bs = 64
data_lm = (TextList.from_df(geoall, LM_PATH, cols='text')
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1))

In [12]:
data_lm.show_batch()

idx,text
0,"lu weesu doon gi mu doonoon kenn ci xxunk ak xxunk yi gën a siiw ci àdduna bi ci taariixu xam - xam , doonoon na xalaatkat , ku bari yëggu - yëggu ci yeneen fànn ( tambalee ca xeltu ba ci politig ) . xxmaj boole nañu ko ci gëstukat yi gën a màgg ci xarnub xxup xx . xxmaj bokk na ci li ko gën a siiwal gisiin"
1,xxmaj pakistaan - xxmaj palau - xxmaj palestiin - xxmaj panamaa - xxmaj xxunk - xxmaj gine - xxmaj gu - xxmaj bees - xxmaj xxunk - xxmaj peru - xxmaj xxunk - xxmaj polineesi gu xxmaj faraas - xxmaj poloñ - xxmaj poortorikoo - xxunk xxmaj xxunk - xxmaj réewum xxmaj cek - xxmaj réewum xxmaj diggu xxmaj afrig - xxmaj réewum xxmaj xxunk - xxmaj réewum xxmaj xxunk
2,"ecoopi moo doonoon menn rèew mi ñu xxunk a teg xxmaj cosaan ak diine . xxmaj xool : xxmaj diiney xxunk . xxmaj xxunk - làkk yi ci xayma nee nañ 2 000 làkk ci goxu xxmaj afrik , ñu boole leen ci ñeenti mbooloo , làkk yi fa xxunk xxunk ci . xxmaj xool it . xxmaj koom - koomu xxmaj afrig xxbos réewu xxmaj afrig xxbos xxmaj aamerig"
3,"xxunk "" or "" xxmaj xxunk "" ; xxmaj ci faranse mooy "" xxunk xxmaj injiil dañuy gis ñaari nit ñi bokk tur "" xxmaj xxunk "" . xxbos xxmaj ci làkku ibrë la tur wi jóge . xxmaj ci angale mooy "" xxmaj xxunk "" ; xxmaj ci faranse mooy "" xxunk bu mag ba la woon , ci jamano nguuru xxmaj daawuda . xxmaj man nañu ko gis"
4,"woon ne xxmaj dafëne nit la woon , ki xxmaj apolo , benn yàllay xxmaj gereg , soppoon . xxmaj apolo dafa ko dàqe di dàqe ba ci mujj xxmaj dafëne dafa soppiku nekk as xxunk bu tudd ' xxunk ' . xxmaj goxu xxmaj dafëne amoon na benn xxunk walla xxunk bi ñu defaroon ngir bànneex rekk bu am xxunk 16 kilomet . xxmaj ca biir màggalukaayu xxmaj dafëne"


In [128]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0)  # be careful about the path

In [129]:
lr = 1e-3
lr *= bs/48

In [130]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(8, lr, moms=(0.8,0.7))  # turn 1 to 8

epoch,train_loss,valid_loss,accuracy,time
0,5.355139,4.560729,0.326451,08:07
1,4.993589,4.224590,0.349442,07:43
2,4.685068,4.001536,0.355134,08:36
3,4.387579,3.763090,0.354492,08:57
4,4.087812,3.475447,0.354492,07:52
5,3.822635,3.340496,0.361189,08:53
6,3.672056,3.302410,0.365485,08:30
7,3.614186,3.296517,0.365904,08:48


In [131]:
learn_lm.save(f'{lang}fine_tuned')
learn_lm.save_encoder(f'{lang}fine_tuned_enc')

# build a classifier

In [13]:
data_clas = (TextList.from_df(geoall, LM_PATH, vocab=data_lm.vocab, cols='text')
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols='geo')
    .databunch(bs=bs, num_workers=1))

data_clas.save(f'{lang}_textlist_class')

In [14]:
data_clas = load_data(LM_PATH, f'{lang}_textlist_class', bs=bs, num_workers=1)

In [37]:
from sklearn.metrics import f1_score
def f1(inp,targ): return f1_score(targ, np.argmax(inp, axis=-1))

In [23]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn_c.load_encoder(f'{lang}fine_tuned_enc')
learn_c.freeze()

In [24]:
lr=2e-2
lr *= bs/48

In [25]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.522542,0.429010,0.917910,07:56
1,0.449735,0.277386,0.902985,06:45


In [26]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.412928,0.284921,0.880597,08:03
1,0.366290,0.236806,0.917910,06:50


In [27]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.271405,0.326164,0.910448,12:00
1,0.265042,0.222919,0.932836,13:03


In [28]:
learn_c.unfreeze()
learn_c.fit_one_cycle(1, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.213600,0.226905,0.932836,13:56


In [29]:
learn_c.save(f'{lang}clas')

# prediction

In [30]:
test_df.iloc[16]['text']

'Northern Mariana Islands ("Sankattan Siha Na Islas Mariånas", Komonwels bu Iil yu Northern Mariana).'

In [31]:
learn_c.predict(test_df.iloc[16]['text'])

(Category 0, tensor(0), tensor([0.9815, 0.0185]))

In [39]:
preds,targs = learn_c.get_preds(ordered=True)
accuracy(preds,targs),f1(preds,targs)

(tensor(0.9328), 0.8421052631578947)